# Simple Movie Recommender System using LensKit library

Instruction by Crash Course AI, Lab #4

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import lenskit as lk

In [8]:
!git clone https://github.com/crash-course-ai/lab4-recommender-systems
data = lk.datasets.MovieLens('lab4-recommender-systems')

Cloning into 'lab4-recommender-systems'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 25 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.


In [10]:
data.ratings.head(10)

,user,item,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [36]:
data.movies.head(10)

,title,genres
item,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action


In [11]:
joined = data.ratings.join(data.movies['genres'], on='item')
joined = joined.join(data.movies['title'], on='item')
joined.head(10)

,user,item,rating,timestamp,genres,title
0,1,1,4.0,964982703,Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995)
1,1,3,4.0,964981247,Comedy|Romance,Grumpier Old Men (1995)
2,1,6,4.0,964982224,Action|Crime|Thriller,Heat (1995)
3,1,47,5.0,964983815,Mystery|Thriller,Seven (a.k.a. Se7en) (1995)
4,1,50,5.0,964982931,Crime|Mystery|Thriller,"Usual Suspects, The (1995)"
5,1,70,3.0,964982400,Action|Comedy|Horror|Thriller,From Dusk Till Dawn (1996)
6,1,101,5.0,964980868,Adventure|Comedy|Crime|Romance,Bottle Rocket (1996)
7,1,110,4.0,964982176,Action|Drama|War,Braveheart (1995)
8,1,151,5.0,964984041,Action|Drama|Romance|War,Rob Roy (1995)
9,1,157,5.0,964984100,Comedy|War,Canadian Bacon (1995)


In [15]:
avg_rt = (data.ratings).groupby('item').mean().sort_values(by='rating', ascending=False)
joined = avg_rt.join(data.movies['title'], on='item')
joined = joined.join(data.movies['genres'], on='item')
joined.head()

,user,rating,timestamp,title,genres
item,,,,,
88448,483.0,5.0,1.315438e+09,Paper Birds (Pájaros de papel) (2010),Comedy|Drama
100556,462.0,5.0,1.456151e+09,"Act of Killing, The (2012)",Documentary
143031,89.0,5.0,1.520409e+09,Jump In! (2007),Comedy|Drama|Romance
143511,105.0,5.0,1.526207e+09,Human (2015),Documentary
143559,89.0,5.0,1.520410e+09,L.A. Slasher (2015),Comedy|Crime|Fantasy


In [19]:
avg_rt = data.ratings.groupby('item').agg(count=('user', 'size'), rating=('rating', 'mean'))
joined = avg_rt.join(data.movies['title'], on='item')
joined = joined.join(data.movies['genres'], on='item')
joined.head(10)

,count,rating,title,genres
item,,,,
1,215,3.920930,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,110,3.431818,Jumanji (1995),Adventure|Children|Fantasy
3,52,3.259615,Grumpier Old Men (1995),Comedy|Romance
4,7,2.357143,Waiting to Exhale (1995),Comedy|Drama|Romance
5,49,3.071429,Father of the Bride Part II (1995),Comedy
6,102,3.946078,Heat (1995),Action|Crime|Thriller
7,54,3.185185,Sabrina (1995),Comedy|Romance
8,8,2.875000,Tom and Huck (1995),Adventure|Children
9,16,3.125000,Sudden Death (1995),Action


In [24]:
avg_rt = (data.ratings).groupby('item').agg(count=('user', 'size'), rating=('rating', 'mean')) \
        .reset_index() \
        .sort_values(by='rating', ascending=False)
joined = avg_rt.join(data.movies['title'], on='item')
joined = joined.join(data.movies['genres'], on='item')
joined.head(10)

,item,count,rating,title,genres
7638,88448,1,5.0,Paper Birds (Pájaros de papel) (2010),Comedy|Drama
8089,100556,1,5.0,"Act of Killing, The (2012)",Documentary
9065,143031,1,5.0,Jump In! (2007),Comedy|Drama|Romance
9076,143511,1,5.0,Human (2015),Documentary
9078,143559,1,5.0,L.A. Slasher (2015),Comedy|Crime|Fantasy
4245,6201,1,5.0,Lady Jane (1986),Drama|Romance
8136,102217,1,5.0,Bill Hicks: Revelations (1993),Comedy
8130,102084,1,5.0,Justice League: Doom (2012),Action|Animation|Fantasy
4240,6192,1,5.0,Open Hearts (Elsker dig for evigt) (2002),Romance
9104,145994,1,5.0,Formula of Love (1984),Comedy


In [27]:
joined = avg_rt.loc[avg_rt['count']>20].join(data.movies['title'], on='item')
joined = joined.join(data.movies['genres'], on='item')
joined.head(10)

,item,count,rating,title,genres
277,318,317,4.429022,"Shawshank Redemption, The (1994)",Crime|Drama
704,922,27,4.333333,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Drama|Film-Noir|Romance
680,898,29,4.310345,"Philadelphia Story, The (1940)",Comedy|Drama|Romance
905,1204,45,4.300000,Lawrence of Arabia (1962),Adventure|Drama|War
413,475,25,4.300000,In the Name of the Father (1993),Drama
210,246,29,4.293103,Hoop Dreams (1994),Documentary
659,858,192,4.289062,"Godfather, The (1972)",Crime|Drama
934,1235,26,4.288462,Harold and Maude (1971),Comedy|Drama|Romance
9445,168252,25,4.280000,Logan (2017),Action|Sci-Fi
2224,2959,218,4.272936,Fight Club (1999),Action|Crime|Drama|Thriller


In [77]:
my_rt = pd.read_csv('ratings.csv')
my_rt.drop('Unnamed: 0', axis=1, inplace=True)
my_rt.rename(columns={'titles':'title', 'imdb':'imdbId', 'Rating':'rating'}, inplace=True)
my_rt.imdbId = my_rt.imdbId.apply(lambda x: x[2:])
my_rt.imdbId = my_rt.imdbId.astype('i')
my_rt.head() 

,imdbId,title,slug,rating
0,468569,The Dark Knight,the-dark-knight-2008,10.0
1,12221748,Becoming,becoming-2020-05-06,8.0
2,169547,American Beauty,american-beauty-1999,10.0
3,209144,Memento,memento-2000,6.0
4,75314,Taxi Driver,taxi-driver-1976,8.0


In [89]:
data = lk.datasets.MovieLens('ml-latest-small')
data.links['item'] = data.links.index.values
data.links.item = data.links.item.astype('i')
data.links.set_index('imdbId', inplace=True)
data.links.head()

,tmdbId,item
imdbId,,
114709,862,1
113497,8844,2
113228,15602,3
114885,31357,4
113041,11862,5


In [103]:
cand = my_rt.join(data.links.item, on='imdbId')
cand.loc[cand.item.isna()].shape
cand_dict = cand.dropna()
cand_dict.item = cand_dict.item.astype('i')
cand_dict.head(10)

,imdbId,title,slug,rating,item
0,468569,The Dark Knight,the-dark-knight-2008,10.0,58559
2,169547,American Beauty,american-beauty-1999,10.0,2858
3,209144,Memento,memento-2000,6.0,4226
4,75314,Taxi Driver,taxi-driver-1976,8.0,111
5,68646,The Godfather,the-godfather-1972,10.0,858
6,71562,The Godfather: Part II,the-godfather-part-ii-1974,10.0,1221
7,102926,The Silence of the Lambs,the-silence-of-the-lambs-1991,8.0,593
8,111161,The Shawshank Redemption,the-shawshank-redemption-1994,10.0,318
9,50083,12 Angry Men,12-angry-men-1957,8.0,1203
10,52357,Vertigo,vertigo-1958,8.0,903


In [104]:
cand_dict = dict(zip(cand_dict.item, cand_dict.rating/2))
cand_dict

{58559: 5.0,
 2858: 5.0,
 4226: 3.0,
 111: 4.0,
 858: 5.0,
 1221: 5.0,
 593: 4.0,
 318: 5.0,
 1203: 4.0,
 903: 4.0,
 1219: 5.0,
 5291: 3.5,
 904: 5.0,
 44555: 5.0,
 1721: 3.0,
 3949: 4.0,
 27773: 5.0,
 1960: 5.0,
 44191: 4.0,
 1213: 3.0,
 46578: 5.0,
 7327: 3.0,
 4144: 4.0,
 4848: 4.0,
 48780: 4.0,
 46723: 5.0,
 5225: 5.0,
 48516: 3.0,
 6890: 5.0,
 51540: 3.0,
 50872: 4.0,
 7024: 5.0,
 495: 5.0,
 1280: 4.0,
 69640: 3.0,
 74458: 5.0,
 27869: 3.5,
 7926: 4.0,
 55908: 5.0,
 73808: 4.0,
 66371: 5.0,
 68157: 3.0,
 26131: 4.0,
 69134: 4.0,
 7745: 3.5,
 4334: 5.0,
 76091: 5.0,
 326: 5.0,
 91529: 3.0,
 104841: 4.0,
 82667: 4.5,
 89759: 5.0,
 99114: 3.0,
 96821: 3.0,
 106782: 4.0,
 112556: 3.0,
 133645: 3.0,
 118896: 3.5,
 139385: 3.0,
 158783: 3.0,
 140162: 3.0,
 164909: 2.0,
 142488: 5.0,
 140267: 3.5,
 148626: 4.0,
 163134: 5.0,
 174055: 4.0,
 162082: 3.5,
 162414: 5.0,
 168492: 2.5,
 175303: 2.0,
 180031: 2.0}

In [105]:
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser

In [108]:
num_recs = 15
model = UserUser(15, min_nbrs=3)
algo = Recommender.adapt(model)
algo.fit(data.ratings)

In [113]:
my_recs = algo.recommend(-1, 30, ratings=pd.Series(data=cand_dict))
joined = my_recs.join(data.movies['genres'], on='item')
joined = joined.join(data.movies['title'], on='item')
joined

,item,score,genres,title
0,51931,5.902881,Drama,Reign Over Me (2007)
1,102686,5.542726,Comedy,"Hangover Part III, The (2013)"
2,174053,5.524936,Drama|Horror|Mystery|Sci-Fi|Thriller,Black Mirror: White Christmas (2014)
3,1046,5.447419,Drama|Romance,Beautiful Thing (1996)
4,177593,5.438893,Crime|Drama,"Three Billboards Outside Ebbing, Missouri (2017)"
5,1366,5.308604,Drama,"Crucible, The (1996)"
6,7842,5.292607,Drama|Fantasy|Sci-Fi,Dune (2000)
7,3224,5.292407,Drama,Woman in the Dunes (Suna no onna) (1964)
8,3022,5.290768,Comedy|War,"General, The (1926)"
9,334,5.224060,Drama,Vanya on 42nd Street (1994)


In [112]:
data.links.loc[data.links.item==3224]

,tmdbId,item
imdbId,,
58625,16672,3224


In [115]:
dict({'three': 3, 'one': 1, 'two': 2})

{'three': 3, 'one': 1, 'two': 2}

In [130]:
import requests

In [124]:
data = lk.datasets.MovieLens('ml-latest-small')
data.links.head()

,imdbId,tmdbId
item,,
1,114709,862
2,113497,8844
3,113228,15602
4,114885,31357
5,113041,11862


In [125]:
recs.set_index('item')

,score
item,
51931,5.902881
102686,5.542726
174053,5.524936
1046,5.447419
177593,5.438893
1366,5.308604
7842,5.292607
3224,5.292407
3022,5.290768


In [312]:
recs = joined[['item', 'score']]
recs = recs.join(data.links.imdbId, on='item')
recs = pd.merge(recs, joined.drop('score',axis=1), on='item', how='left')
#recs.imdbId = recs.imdbId.apply(lambda x: 'tt' + str(x).zfill(7))
recs.head()

,item,score,imdbId,genres,title
0,51931,5.902881,490204,Drama,Reign Over Me (2007)
1,102686,5.542726,1951261,Comedy,"Hangover Part III, The (2013)"
2,174053,5.524936,3973198,Drama|Horror|Mystery|Sci-Fi|Thriller,Black Mirror: White Christmas (2014)
3,1046,5.447419,115640,Drama|Romance,Beautiful Thing (1996)
4,177593,5.438893,5027774,Crime|Drama,"Three Billboards Outside Ebbing, Missouri (2017)"


In [282]:
recs_inf[10]

{'Title': "It's the Great Pumpkin, Charlie Brown",
 'Year': '1966',
 'Rated': 'Not Rated',
 'Released': '27 Oct 1966',
 'Runtime': '25 min',
 'Genre': 'Animation, Comedy, Family',
 'Director': 'Bill Melendez',
 'Writer': 'Charles M. Schulz',
 'Actors': 'Peter Robbins, Christopher Shea, Sally Dryer, Cathy Steinberg',
 'Plot': 'The Peanuts gang celebrates Halloween while Linus waits for the Great Pumpkin.',
 'Language': 'English',
 'Country': 'USA',
 'Awards': 'Nominated for 3 Primetime Emmys. Another 1 nomination.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTQzMDc1NTYzM15BMl5BanBnXkFtZTgwMDQ2NDUzMzE@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.2/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '94%'}],
 'Metascore': 'N/A',
 'imdbRating': '8.2',
 'imdbVotes': '15,399',
 'imdbID': 'tt0060550',
 'Type': 'movie',
 'DVD': 'N/A',
 'BoxOffice': 'N/A',
 'Production': 'N/A',
 'Website': 'N/A',
 'Response': 'True'}

In [306]:
recs.drop('imdbRating', axis=1, inplace=True)
recs.insert(12, column='imdbRating', value=np.nan)
recs

,item,score,imdbId,genres,title,Runtime,Director,Actors,Plot,Language,Country,Awards,imdbRating,imdbVotes,RT
0,51931,5.902881,490204,Drama,Reign Over Me (2007),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,102686,5.542726,1951261,Comedy,"Hangover Part III, The (2013)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,174053,5.524936,3973198,Drama|Horror|Mystery|Sci-Fi|Thriller,Black Mirror: White Christmas (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1046,5.447419,115640,Drama|Romance,Beautiful Thing (1996),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,177593,5.438893,5027774,Crime|Drama,"Three Billboards Outside Ebbing, Missouri (2017)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1366,5.308604,115988,Drama,"Crucible, The (1996)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7842,5.292607,142032,Drama|Fantasy|Sci-Fi,Dune (2000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3224,5.292407,58625,Drama,Woman in the Dunes (Suna no onna) (1964),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3022,5.290768,17925,Comedy|War,"General, The (1926)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,334,5.224060,111590,Drama,Vanya on 42nd Street (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [270]:
#for i, r in enumerate(recs_inf[:2]):
    #print(r)
#    recs.iloc[i].loc[:, ['imdbRating']] = r['Ratings'][0]['Value'][:-3]
#    recs.imdbRating = recs.Ratings[0]['Value']
#    recs.RT = recs.Ratings[1]['Value']

In [303]:
df = pd.DataFrame({'angles': [0, 3, 4], 'test': [1, 2, 3],
                   'degrees': [360, 180, 360]},
                  index=['circle', 'triangle', 'rectangle'])
df

,angles,test,degrees
circle,0,1,360
triangle,3,2,180
rectangle,4,3,360


In [305]:
df.loc[0, ['angles', 'test']] = [10, 20]

In [311]:
recs_inf[0].keys()

dict_keys(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster', 'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type', 'DVD', 'BoxOffice', 'Production', 'Website', 'Response'])

In [322]:
col

['Language',
 'Actors',
 'imdbRating',
 'Awards',
 'Plot',
 'Country',
 'imdbVotes',
 'Director',
 'Runtime']

In [468]:
rec_test = recs.copy(deep=True)
rec_test.head()

,item,score,imdbId,genres,title
0,51931,5.902881,490204,Drama,Reign Over Me (2007)
1,102686,5.542726,1951261,Comedy,"Hangover Part III, The (2013)"
2,174053,5.524936,3973198,Drama|Horror|Mystery|Sci-Fi|Thriller,Black Mirror: White Christmas (2014)
3,1046,5.447419,115640,Drama|Romance,Beautiful Thing (1996)
4,177593,5.438893,5027774,Crime|Drama,"Three Billboards Outside Ebbing, Missouri (2017)"


In [469]:
# create columns
columns = ['Runtime', 'Director', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'imdbVotes']
for ind, rec in enumerate(recs_inf):
    for c in columns:
        rec_test.loc[ind, c] = rec[c]
    for rating in rec['Ratings']:
        rec_test.loc[ind, rating['Source']] = rating['Value']
rec_test

,item,score,imdbId,genres,title,Runtime,Director,Actors,Plot,Language,Country,Awards,imdbVotes,Internet Movie Database,Rotten Tomatoes,Metacritic
0,51931,5.902881,490204,Drama,Reign Over Me (2007),124 min,Mike Binder,"Adam Sandler, Don Cheadle, Jada Pinkett Smith,...",A man who lost his family in the September 11 ...,English,USA,3 nominations.,"91,572",7.4/10,64%,61/100
1,102686,5.542726,1951261,Comedy,"Hangover Part III, The (2013)",100 min,Todd Phillips,"Bradley Cooper, Ed Helms, Zach Galifianakis, J...",When one of their own is kidnapped by an angry...,"English, Spanish",USA,2 wins & 7 nominations.,"283,880",5.8/10,20%,30/100
2,174053,5.524936,3973198,Drama|Horror|Mystery|Sci-Fi|Thriller,Black Mirror: White Christmas (2014),73 min,Carl Tibbetts,"Jon Hamm, Rafe Spall, Oona Chaplin, Natalia Tena",Three interconnected tales of technology run a...,English,UK,N/A,44154,9.2/10,NaN,NaN
3,1046,5.447419,115640,Drama|Romance,Beautiful Thing (1996),90 min,Hettie Macdonald,"Linda Henry, Glen Berry, Scott Neal, Tameka Em...","In a suburb of London, young Jamie is escaping...",English,UK,4 wins & 3 nominations.,"21,907",7.5/10,91%,NaN
4,177593,5.438893,5027774,Crime|Drama,"Three Billboards Outside Ebbing, Missouri (2017)",115 min,Martin McDonagh,"Frances McDormand, Caleb Landry Jones, Kerry C...",A mother personally challenges the local autho...,English,"UK, USA",Won 2 Oscars. Another 130 wins & 225 nominations.,"394,263",8.2/10,90%,88/100
5,1366,5.308604,115988,Drama,"Crucible, The (1996)",124 min,Nicholas Hytner,"Daniel Day-Lewis, Winona Ryder, Paul Scofield,...",A Salem resident attempts to frame her ex-love...,English,USA,Nominated for 2 Oscars. Another 4 wins & 27 no...,"34,109",6.8/10,68%,NaN
6,7842,5.292607,142032,Drama|Fantasy|Sci-Fi,Dune (2000),265 min,N/A,"William Hurt, Alec Newman, Saskia Reeves, P.H....","A three-part miniseries on politics, betrayal,...","English, Italian","USA, Canada, Germany, Italy",Won 2 Primetime Emmys. Another 7 wins & 9 nomi...,"17,574",7.1/10,NaN,NaN
7,3224,5.292407,58625,Drama,Woman in the Dunes (Suna no onna) (1964),123 min,Hiroshi Teshigahara,"Eiji Okada, Kyôko Kishida, Hiroko Itô, Kôji Mi...",An entomologist on vacation is trapped by loca...,Japanese,Japan,Nominated for 2 Oscars. Another 10 wins & 1 no...,"16,002",8.5/10,100%,NaN
8,3022,5.290768,17925,Comedy|War,"General, The (1926)",67 min,"Clyde Bruckman, Buster Keaton","Marion Mack, Glen Cavender, Jim Farley, Freder...",When Union spies steal an engineer's beloved l...,English,USA,2 wins & 1 nomination.,"76,801",8.1/10,93%,NaN
9,334,5.224060,111590,Drama,Vanya on 42nd Street (1994),119 min,Louis Malle,"Phoebe Brand, Lynn Cohen, George Gaynes, Jerry...","New York actors rehearse Chekhov's ""Uncle Vany...",English,USA,2 wins & 12 nominations.,"4,029",7.4/10,89%,NaN


In [338]:
key='Director'
dict({key:recs_inf[0][key]})

{'Director': 'Mike Binder'}

In [510]:
recs.imdbVotes = recs.imdbVotes.apply(lambda x: x.replace(',', '')).astype('i')
recs.sort_values(by='imdbVotes', ascending=False)

,item,score,imdbId,genres,title,Runtime,Director,Actors,Plot,Language,Country,Awards,imdbVotes,Internet Movie Database,Rotten Tomatoes,Metacritic
23,171763,5.091117,3890160,Action|Crime|Thriller,Baby Driver (2017),113 min,Edgar Wright,"Ansel Elgort, Jon Bernthal, Jon Hamm, Eiza Gon...",After being coerced into working for a crime b...,"English, American Sign Language","UK, USA",Nominated for 3 Oscars. Another 41 wins & 61 n...,406935,7.6/10,93%,86/100
4,177593,5.438893,5027774,Crime|Drama,"Three Billboards Outside Ebbing, Missouri (2017)",115 min,Martin McDonagh,"Frances McDormand, Caleb Landry Jones, Kerry C...",A mother personally challenges the local autho...,English,"UK, USA",Won 2 Oscars. Another 130 wins & 225 nominations.,394263,8.2/10,90%,88/100
1,102686,5.542726,1951261,Comedy,"Hangover Part III, The (2013)",100 min,Todd Phillips,"Bradley Cooper, Ed Helms, Zach Galifianakis, J...",When one of their own is kidnapped by an angry...,"English, Spanish",USA,2 wins & 7 nominations.,283880,5.8/10,20%,30/100
26,89904,5.071850,1655442,Comedy|Drama|Romance,The Artist (2011),100 min,Michel Hazanavicius,"Jean Dujardin, Bérénice Bejo, John Goodman, Ja...",An egomaniacal film star develops a relationsh...,"English, French","France, Belgium, USA",Won 5 Oscars. Another 156 wins & 203 nominations.,224074,7.9/10,95%,89/100
12,68952,5.193958,1127180,Comedy|Horror,Drag Me to Hell (2009),99 min,Sam Raimi,"Alison Lohman, Justin Long, Lorna Raver, Dilee...",A loan officer who evicts an old woman from he...,"English, Spanish, Hungarian, Czech",USA,7 wins & 22 nominations.,182901,6.5/10,92%,83/100
13,8042,5.191751,70379,Crime|Drama,Mean Streets (1973),112 min,Martin Scorsese,"Robert De Niro, Harvey Keitel, David Proval, A...",A small-time hood aspires to work his way up t...,"English, Italian",USA,5 wins & 5 nominations.,93577,7.3/10,97%,96/100
0,51931,5.902881,490204,Drama,Reign Over Me (2007),124 min,Mike Binder,"Adam Sandler, Don Cheadle, Jada Pinkett Smith,...",A man who lost his family in the September 11 ...,English,USA,3 nominations.,91572,7.4/10,64%,61/100
28,94070,5.066953,1412386,Comedy|Drama,"Best Exotic Marigold Hotel, The (2011)",124 min,John Madden,"Judi Dench, Tom Wilkinson, Patrick Pearson, Hu...",British retirees travel to India to take up re...,"English, Hindi","UK, USA, United Arab Emirates",Nominated for 2 Golden Globes. Another 4 wins ...,87886,7.2/10,79%,62/100
8,3022,5.290768,17925,Comedy|War,"General, The (1926)",67 min,"Clyde Bruckman, Buster Keaton","Marion Mack, Glen Cavender, Jim Farley, Freder...",When Union spies steal an engineer's beloved l...,English,USA,2 wins & 1 nomination.,76801,8.1/10,93%,NaN
22,62293,5.092698,864761,Drama|Romance,"Duchess, The (2008)",110 min,Saul Dibb,"Keira Knightley, Ralph Fiennes, Charlotte Ramp...",A chronicle of the life of 18th-century aristo...,English,"UK, Italy, France, USA",Won 1 Oscar. Another 7 wins & 21 nominations.,75837,6.9/10,62%,62/100
